## FINAL VERSION Segmenting & Clustering Neighbourhoods in Toronto Assignment

Download required dependencies

In [1]:
#install Beautiful Soup, requests & lmxl for Web Scaping
!pip install BeautifulSoup4
!pip install requests
!pip install lxml

In [42]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

<img src="Toronto_df.jpg">

### To create the above dataframe:

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [14]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df=pd.read_html(url, header=0)[0]

df.rename(columns={'Postal Code':'Postalcode'},inplace=True)

df = df[df.Borough != 'Not assigned'].reset_index(drop=True)

df = df.groupby(['Postalcode','Borough'], as_index=False).agg(lambda x: ','.join(x))

mask = df['Neighbourhood'] == "Not assigned"
df.loc[mask, 'Neighbourhood'] = df.loc[mask, 'Borough']
df.head()

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [4]:
df.shape

(103, 3)

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:

<img src="geocode.png">

Use the Geocoder package or the csv file to create the following dataframe:

<img src="latlong.png">

In [5]:
!pip install geocoder

In [6]:
import geocoder # import geocoder

def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [9]:
df_geo=pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
df_geo.rename(columns={'Postal Code':'Postalcode'},inplace=True)
df_merged= pd.merge(df_geo, df, on='Postalcode')
df_merged=df_merged[['Postalcode','Borough','Neighbourhood','Latitude','Longitude']]

In [17]:
df_merged.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

#### Return only records with Toronto in their Borough

In [20]:
df_tor = df_merged[df_merged['Borough'].str.contains('Toronto',regex=False)]
df_tor.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [22]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    brotlipy-0.7.0             |py36h8c4c3a4_1000         346 KB  conda-forge
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    cryptography-3.0           |   py36h45558ae_0         640 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    pandas-1.1.0               |   py36h831f99a_0        10.5 MB  conda-

In [29]:
tor_map = folium.Map(location=[43.6532, -79.3832],zoom_start=11)

for lat,lng,borough,neighbourhood in zip(df_tor['Latitude'],df_tor['Longitude'],df_tor['Borough'],df_tor['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='red',
    fill=True,
    fill_color='#31ccc5',
    fill_opacity=0.5,
    parse_html=False).add_to(tor_map)
tor_map

### Use KMeans to cluster the neighbourhoods

In [31]:
# set number of clusters
kclusters = 5

tor_clustering = df_tor.drop(['Postalcode','Borough','Neighbourhood'],1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1], dtype=int32)

In [36]:
df_tor.insert(0,'Cluster Labels', kmeans.labels_)

,Cluster Labels,Postalcode,Borough,Neighbourhood,Latitude,Longitude
37,0,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,0,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,0,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,0,M4M,East Toronto,Studio District,43.659526,-79.340923
44,1,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,1,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
47,1,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,1,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,1,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


In [43]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832],zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Neighbourhood'], df_tor['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

### Cluster 1

In [50]:
df_tor.loc[df_tor['Cluster Labels'] == 0, df_tor.columns[[1] + list(range(2, df_tor.shape[1]))]]

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
87,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558


### Cluster 2

In [51]:
df_tor.loc[df_tor['Cluster Labels'] == 1, df_tor.columns[[1] + list(range(2, df_tor.shape[1]))]]

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049
63,M5N,Central Toronto,Roselawn,43.711695,-79.416936
64,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307


#### Cluster 3

In [52]:
df_tor.loc[df_tor['Cluster Labels'] == 2, df_tor.columns[[1] + list(range(2, df_tor.shape[1]))]]

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
76,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
82,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763
83,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
84,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450


### Cluster 4

In [53]:
df_tor.loc[df_tor['Cluster Labels'] == 3, df_tor.columns[[1] + list(range(2, df_tor.shape[1]))]]

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
65,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
66,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049
67,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049
75,M6G,Downtown Toronto,Christie,43.669542,-79.422564
77,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
78,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191


### Cluster 5

In [54]:
df_tor.loc[df_tor['Cluster Labels'] == 4, df_tor.columns[[1] + list(range(2, df_tor.shape[1]))]]

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
51,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
52,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
53,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
54,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
55,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
56,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
58,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
59,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
